In [ ]:
import boto3

client=boto3.client('sts')
account=client.get_caller_identity()['Account']

my_session=boto3.session.Session()
region=my_session.region_name

algorithm_name="pytorch-tft-container-test"
ecr_image='{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

ecr_image

In [2]:
from utils_timeseries import download_process_and_return_raw_data, save_local_and_upload_s3, metadata_json_upload_s3
import sagemaker
import uuid

max_prediction_length = 6
max_encoder_length = 24
special_days = [
        "easter_day",
        "good_friday",
        "new_year",
        "christmas",
        "labor_day",
        "independence_day",
        "revolution_day_memorial",
        "regional_games",
        "fifa_u_17_world_cup",
        "football_gold_cup",
        "beer_capital",
        "music_fest",
    ]

training_metadata = {}
training_metadata['time_idx'] = "time_idx"
training_metadata['target'] = "volume"
training_metadata['group_ids'] = ["agency", "sku"]
training_metadata['min_encoder_length'] = max_encoder_length // 2      # keep encoder length long (as it is in the validation set)
training_metadata['max_encoder_length'] = max_encoder_length
training_metadata['min_prediction_length'] = 1      
training_metadata['max_prediction_length'] = max_prediction_length
training_metadata['static_categoricals'] = ["agency", "sku"]
training_metadata['static_reals'] = ["avg_population_2017", "avg_yearly_household_income_2017"]
training_metadata['time_varying_known_categoricals'] = ["special_days", "month"]
training_metadata['variable_groups'] = {"special_days": special_days}
training_metadata['time_varying_known_reals'] = ["time_idx", "price_regular", "discount_in_percent"]
training_metadata['time_varying_unknown_categoricals'] = []
training_metadata['time_varying_unknown_reals'] = [
        "volume",
        "log_volume",
        "industry_volume",
        "soda_volume",
        "avg_max_temp",
        "avg_volume_by_agency",
        "avg_volume_by_sku",
    ]
training_metadata['target_normalizer'] = {
                            "normalized_groups": ["agency", "sku"],
                            "normalization_transformation": 'softplus'
                        }
training_metadata['add_relative_time_idx'] = True
training_metadata['add_target_scales'] = True
training_metadata['add_encoder_length'] = True




sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

# obtain & upload training data
training_data = download_process_and_return_raw_data()
inputs = save_local_and_upload_s3(training_data, sagemaker_session, bucket, data_filename="stallion_data")

# upload metadata
training_metadata['training_cutoff'] = int(training_data["time_idx"].max() - max_prediction_length)
metadata_json_upload_s3(training_metadata, sagemaker_session, bucket, metadata_filename="stallion_metadata")

Checkpointing directory timeseries_data exists
saved raw data to timeseries_data/stallion_data.parquet
Checkpointing directory timeseries_data exists
saved metadata to timeseries_data/stallion_metadata.json


's3://sagemaker-us-east-1-551329315830/data/timeseries_data'

In [ ]:
import sagemaker

from sagemaker import get_execution_role
from sagemaker.estimator import Estimator

estimator=Estimator(
    image_uri=ecr_image,
    role=get_execution_role(),
    base_job_name=algorithm_name,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    input_mode = "Pipe"
)

# start training
estimator.fit(inputs)

# # deploy the trained model
# predictor=estimator.deploy(1, instance_type)